In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('BDCW').getOrCreate()

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

In [3]:
import pyspark.sql.functions as Func
import pyspark.sql.types as Ty

In [4]:
rain_df=spark.read.csv('weatherAUS.csv',header=True,inferSchema=True)
rain_df.show(5)

+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+
|      Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RainTomorrow|
+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+
|2008-12-01|  Albury|   13.4|   22.9|     0.6|         NA|      NA|          W|           44|         W|       WNW|          20|          24|         71|         22|     1007.7|     1007.1|       8|      NA|   16.9|   21.8|       No|          No|
|2008-12-02|

In [5]:
print(rain_df.columns)

['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm', 'RainToday', 'RainTomorrow']


In [6]:
rain_df.count()

145460

In [7]:
#replacing 'NA' values with null 
from pyspark.sql.functions import when, lit, col

def replace(column, value):
    return when(column != value, column).otherwise(lit(None))

rain_df = rain_df.withColumn("Date",replace(col("Date"), "NA"))
rain_df = rain_df.withColumn("Location",replace(col("Location"), "NA"))
rain_df = rain_df.withColumn("MinTemp",replace(col("MinTemp"), "NA"))
rain_df = rain_df.withColumn("MaxTemp",replace(col("MaxTemp"), "NA"))
rain_df = rain_df.withColumn("Rainfall",replace(col("Rainfall"), "NA"))
rain_df = rain_df.withColumn("Evaporation",replace(col("Evaporation"), "NA"))
rain_df = rain_df.withColumn("Sunshine",replace(col("Sunshine"), "NA"))
rain_df = rain_df.withColumn("WindGustDir",replace(col("WindGustDir"), "NA"))
rain_df = rain_df.withColumn("WindGustSpeed",replace(col("WindGustSpeed"), "NA"))
rain_df = rain_df.withColumn("WindDir9am",replace(col("WindDir9am"), "NA"))
rain_df = rain_df.withColumn("WindDir3pm",replace(col("WindDir3pm"), "NA"))
rain_df = rain_df.withColumn("WindSpeed9am",replace(col("WindSpeed9am"), "NA"))
rain_df = rain_df.withColumn("WindSpeed3pm",replace(col("WindSpeed3pm"), "NA"))
rain_df = rain_df.withColumn("Humidity9am",replace(col("Humidity9am"), "NA"))
rain_df = rain_df.withColumn("Humidity3pm",replace(col("Humidity3pm"), "NA"))
rain_df = rain_df.withColumn("Pressure9am",replace(col("Pressure9am"), "NA"))
rain_df = rain_df.withColumn("Pressure3pm",replace(col("Pressure3pm"), "NA"))
rain_df = rain_df.withColumn("Cloud9am",replace(col("Cloud9am"), "NA"))
rain_df = rain_df.withColumn("Cloud3pm",replace(col("Cloud3pm"), "NA"))
rain_df = rain_df.withColumn("Temp9am",replace(col("Temp9am"), "NA"))
rain_df = rain_df.withColumn("Temp3pm",replace(col("Temp3pm"), "NA"))
rain_df = rain_df.withColumn("RainToday",replace(col("RainToday"), "NA"))
rain_df = rain_df.withColumn("RainTomorrow",replace(col("RainTomorrow"), "NA"))

rain_df.show(5)  

+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+
|      Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RainTomorrow|
+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+
|2008-12-01|  Albury|   13.4|   22.9|     0.6|       null|    null|          W|           44|         W|       WNW|          20|          24|         71|         22|     1007.7|     1007.1|       8|    null|   16.9|   21.8|       No|          No|
|2008-12-02|

In [8]:
#counting the number of null values in each column
from pyspark.sql.functions import isnull, when, count, col

rain_df.select([count(when(isnull(c), c)).alias(c) for c in rain_df.columns]).show()

+----+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+
|Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RainTomorrow|
+----+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+
|   0|       0|   1485|   1261|    3261|      62790|   69835|      10326|        10263|     10566|      4228|        1767|        3062|       2654|       4507|      15065|      15028|   55888|   59358|   1767|   3609|     3261|        3267|
+----+--------+-------+-------+-----

In [9]:
#dropping columns with high null values
rain_df=rain_df.drop('Evaporation', 'Sunshine','Cloud9am','Cloud3pm')

In [10]:
rain_df.columns


['Date',
 'Location',
 'MinTemp',
 'MaxTemp',
 'Rainfall',
 'WindGustDir',
 'WindGustSpeed',
 'WindDir9am',
 'WindDir3pm',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'Temp9am',
 'Temp3pm',
 'RainToday',
 'RainTomorrow']

In [11]:
rain_df.select([count(when(isnull(c), c)).alias(c) for c in rain_df.columns]).show()
print(rain_df.count())

+----+--------+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+-------+-------+---------+------------+
|Date|Location|MinTemp|MaxTemp|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Temp9am|Temp3pm|RainToday|RainTomorrow|
+----+--------+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+-------+-------+---------+------------+
|   0|       0|   1485|   1261|    3261|      10326|        10263|     10566|      4228|        1767|        3062|       2654|       4507|      15065|      15028|   1767|   3609|     3261|        3267|
+----+--------+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+-------+-------+---------+----

In [12]:
rain_df = rain_df.na.drop(subset=["RainTomorrow","RainToday","WindGustDir","WindDir3pm","WindDir9am"])
print(rain_df.count())
rain_df.select([count(when(isnull(c), c)).alias(c) for c in rain_df.columns]).show()

123710
+----+--------+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+-------+-------+---------+------------+
|Date|Location|MinTemp|MaxTemp|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Temp9am|Temp3pm|RainToday|RainTomorrow|
+----+--------+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+-------+-------+---------+------------+
|   0|       0|     90|     45|       0|          0|            0|         0|         0|           0|           0|       1012|       1522|       9120|       9069|    343|    795|        0|           0|
+----+--------+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+-------+-------+-------

In [13]:
rain_df.groupBy('Location').count().show(100)

+----------------+-----+
|        Location|count|
+----------------+-----+
|          Cairns| 2900|
|   NorfolkIsland| 2868|
|         Bendigo| 2725|
|         Walpole| 2506|
|        Canberra| 2762|
|         Woomera| 2917|
|        Adelaide| 2746|
|           Cobar| 2925|
|   SydneyAirport| 2941|
|    PerthAirport| 2924|
|      Wollongong| 2817|
|     Williamtown| 2203|
|           Moree| 2635|
|         Mildura| 2903|
|        Portland| 2890|
|      SalmonGums| 2865|
|        Brisbane| 3029|
|          Sydney| 2285|
|           Perth| 3049|
|            Sale| 2771|
|   BadgerysCreek| 2458|
|          Hobart| 3106|
|     Tuggeranong| 2322|
|        Ballarat| 2895|
|       GoldCoast| 2832|
|MelbourneAirport| 2943|
|        Dartmoor| 2418|
|            Nhil| 1518|
|      PearceRAAF| 2469|
|          Albury| 2445|
|     Witchcliffe| 2653|
|      WaggaWagga| 2801|
|          Darwin| 3119|
|           Uluru| 1446|
|       Nuriootpa| 2812|
|    CoffsHarbour| 2568|
|       Melbourne| 2240|


In [15]:
res  = rain_df.select(*[Func.mean(c).alias(c) for c in rain_df.columns])
res.show()
lak = res.select('MinTemp')
print(lak)

+----+--------+------------------+------------------+------------------+-----------+-----------------+----------+----------+------------------+-----------------+-----------------+----------------+------------------+------------------+------------------+------------------+---------+------------+
|Date|Location|           MinTemp|           MaxTemp|          Rainfall|WindGustDir|    WindGustSpeed|WindDir9am|WindDir3pm|      WindSpeed9am|     WindSpeed3pm|      Humidity9am|     Humidity3pm|       Pressure9am|       Pressure3pm|           Temp9am|           Temp3pm|RainToday|RainTomorrow|
+----+--------+------------------+------------------+------------------+-----------+-----------------+----------+----------+------------------+-----------------+-----------------+----------------+------------------+------------------+------------------+------------------+---------+------------+
|null|    null|12.414077010192518|23.474899931265913|2.3756608196588807|       null|40.68528009053431|      null

In [16]:
from pyspark.sql.types import IntegerType
rain_df=rain_df.withColumn("MinTemp",rain_df.MinTemp.cast(IntegerType()))
rain_df=rain_df.withColumn("MaxTemp",rain_df.MaxTemp.cast(IntegerType()))
rain_df=rain_df.withColumn("Humidity9am",rain_df.Humidity9am.cast(IntegerType()))
rain_df=rain_df.withColumn("Humidity3pm",rain_df.Humidity3pm.cast(IntegerType()))
rain_df=rain_df.withColumn("Temp9am",rain_df.Temp9am.cast(IntegerType()))
rain_df=rain_df.withColumn("Pressure9am",rain_df.Pressure9am.cast(IntegerType()))
rain_df=rain_df.withColumn("Pressure3pm",rain_df.Pressure3pm.cast(IntegerType()))
rain_df=rain_df.withColumn("Temp3pm",rain_df.Temp3pm.cast(IntegerType()))
rain_df=rain_df.withColumn("Rainfall",rain_df.Rainfall.cast(IntegerType()))
rain_df=rain_df.withColumn("WindGustSpeed",rain_df.WindGustSpeed.cast(IntegerType()))
rain_df=rain_df.withColumn("WindSpeed9am",rain_df.WindSpeed9am.cast(IntegerType()))
rain_df=rain_df.withColumn("WindSpeed3pm",rain_df.WindSpeed3pm.cast(IntegerType()))



rain_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: integer (nullable = true)
 |-- MaxTemp: integer (nullable = true)
 |-- Rainfall: integer (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: integer (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: integer (nullable = true)
 |-- WindSpeed3pm: integer (nullable = true)
 |-- Humidity9am: integer (nullable = true)
 |-- Humidity3pm: integer (nullable = true)
 |-- Pressure9am: integer (nullable = true)
 |-- Pressure3pm: integer (nullable = true)
 |-- Temp9am: integer (nullable = true)
 |-- Temp3pm: integer (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RainTomorrow: string (nullable = true)



In [17]:
rain_df.show()

+----------+-------------+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+-------+-------+---------+------------+
|      Date|     Location|MinTemp|MaxTemp|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Temp9am|Temp3pm|RainToday|RainTomorrow|
+----------+-------------+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+-------+-------+---------+------------+
|2009-09-30|       Albury|      4|     21|       0|        NNW|           35|         E|       NNE|           7|          17|         75|         36|       1018|       1013|     11|     20|       No|          No|
|2013-01-10|       Albury|     11|     32|       0|        NNE|           31|        SE|       SSE|           9|          11|         48|         24

In [18]:
from pyspark.ml.feature import Imputer
imputer = Imputer(
    inputCols=['MinTemp','MaxTemp','Humidity9am','Humidity3pm','Pressure9am','Pressure3pm','Temp9am','Temp3pm'], 
    outputCols=["{}_imputed".format(c) for c in ['MinTemp','MaxTemp','Humidity9am','Humidity3pm','Pressure9am',
                                                 'Pressure3pm','Temp9am','Temp3pm']]
    ).setStrategy("mean")
rain_df = imputer.fit(rain_df).transform(rain_df)
rain_df.select([count(when(isnull(c), c)).alias(c) for c in rain_df.columns]).show()

+----+--------+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+-------+-------+---------+------------+---------------+---------------+---------------+-------------------+---------------+-------------------+-------------------+-------------------+
|Date|Location|MinTemp|MaxTemp|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Temp9am|Temp3pm|RainToday|RainTomorrow|MinTemp_imputed|MaxTemp_imputed|Temp9am_imputed|Pressure9am_imputed|Temp3pm_imputed|Humidity3pm_imputed|Humidity9am_imputed|Pressure3pm_imputed|
+----+--------+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+-------+-------+---------+------------+---------------+---------------+---------------+-------------------+---------------+-------------------+---

In [19]:
rain_df.show(100)

+----------+-------------+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+-------+-------+---------+------------+---------------+---------------+---------------+-------------------+---------------+-------------------+-------------------+-------------------+
|      Date|     Location|MinTemp|MaxTemp|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Temp9am|Temp3pm|RainToday|RainTomorrow|MinTemp_imputed|MaxTemp_imputed|Temp9am_imputed|Pressure9am_imputed|Temp3pm_imputed|Humidity3pm_imputed|Humidity9am_imputed|Pressure3pm_imputed|
+----------+-------------+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+-------+-------+---------+------------+---------------+---------------+---------------+-------------------+------

In [20]:
rain_df=rain_df.drop('MinTemp','MaxTemp','Humidity9am','Humidity3pm','Pressure9am','Pressure3pm','Temp9am','Temp3pm')
rain_df.show(100)

+----------+-------------+--------+-----------+-------------+----------+----------+------------+------------+---------+------------+---------------+---------------+---------------+-------------------+---------------+-------------------+-------------------+-------------------+
|      Date|     Location|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|RainToday|RainTomorrow|MinTemp_imputed|MaxTemp_imputed|Temp9am_imputed|Pressure9am_imputed|Temp3pm_imputed|Humidity3pm_imputed|Humidity9am_imputed|Pressure3pm_imputed|
+----------+-------------+--------+-----------+-------------+----------+----------+------------+------------+---------+------------+---------------+---------------+---------------+-------------------+---------------+-------------------+-------------------+-------------------+
|2009-09-30|       Albury|       0|        NNW|           35|         E|       NNE|           7|          17|       No|          No|              4|             21|     

In [21]:
rain_df.select([count(when(isnull(c), c)).alias(c) for c in rain_df.columns]).show()

+----+--------+--------+-----------+-------------+----------+----------+------------+------------+---------+------------+---------------+---------------+---------------+-------------------+---------------+-------------------+-------------------+-------------------+
|Date|Location|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|RainToday|RainTomorrow|MinTemp_imputed|MaxTemp_imputed|Temp9am_imputed|Pressure9am_imputed|Temp3pm_imputed|Humidity3pm_imputed|Humidity9am_imputed|Pressure3pm_imputed|
+----+--------+--------+-----------+-------------+----------+----------+------------+------------+---------+------------+---------------+---------------+---------------+-------------------+---------------+-------------------+-------------------+-------------------+
|   0|       0|       0|          0|            0|         0|         0|           0|           0|        0|           0|              0|              0|              0|                  0|             

In [22]:
print(rain_df.count())

123710


In [23]:
split_date=pyspark.sql.functions.split(rain_df['Date'], '-')     
rain_df= rain_df.withColumn('Year', split_date.getItem(0))
rain_df= rain_df.withColumn('Month', split_date.getItem(1))
rain_df= rain_df.withColumn('Day', split_date.getItem(2))
rain_df=rain_df.drop('Date')
rain_df.show(10)

+-------------+--------+-----------+-------------+----------+----------+------------+------------+---------+------------+---------------+---------------+---------------+-------------------+---------------+-------------------+-------------------+-------------------+----+-----+---+
|     Location|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|RainToday|RainTomorrow|MinTemp_imputed|MaxTemp_imputed|Temp9am_imputed|Pressure9am_imputed|Temp3pm_imputed|Humidity3pm_imputed|Humidity9am_imputed|Pressure3pm_imputed|Year|Month|Day|
+-------------+--------+-----------+-------------+----------+----------+------------+------------+---------+------------+---------------+---------------+---------------+-------------------+---------------+-------------------+-------------------+-------------------+----+-----+---+
|       Albury|       0|        NNW|           35|         E|       NNE|           7|          17|       No|          No|              4|             21|    

In [24]:
rain_df.dtypes

[('Location', 'string'),
 ('Rainfall', 'int'),
 ('WindGustDir', 'string'),
 ('WindGustSpeed', 'int'),
 ('WindDir9am', 'string'),
 ('WindDir3pm', 'string'),
 ('WindSpeed9am', 'int'),
 ('WindSpeed3pm', 'int'),
 ('RainToday', 'string'),
 ('RainTomorrow', 'string'),
 ('MinTemp_imputed', 'int'),
 ('MaxTemp_imputed', 'int'),
 ('Temp9am_imputed', 'int'),
 ('Pressure9am_imputed', 'int'),
 ('Temp3pm_imputed', 'int'),
 ('Humidity3pm_imputed', 'int'),
 ('Humidity9am_imputed', 'int'),
 ('Pressure3pm_imputed', 'int'),
 ('Year', 'string'),
 ('Month', 'string'),
 ('Day', 'string')]

In [25]:
rain_df=rain_df.drop('RainToday')
rain_df.show(10)

+-------------+--------+-----------+-------------+----------+----------+------------+------------+------------+---------------+---------------+---------------+-------------------+---------------+-------------------+-------------------+-------------------+----+-----+---+
|     Location|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|RainTomorrow|MinTemp_imputed|MaxTemp_imputed|Temp9am_imputed|Pressure9am_imputed|Temp3pm_imputed|Humidity3pm_imputed|Humidity9am_imputed|Pressure3pm_imputed|Year|Month|Day|
+-------------+--------+-----------+-------------+----------+----------+------------+------------+------------+---------------+---------------+---------------+-------------------+---------------+-------------------+-------------------+-------------------+----+-----+---+
|       Albury|       0|        NNW|           35|         E|       NNE|           7|          17|          No|              4|             21|             11|               1018|        

In [26]:
numericColumns = [item[0] for item in rain_df.dtypes if item[1].startswith('int') ]
categoricalColumns = [item[0] for item in rain_df.dtypes if item[1].startswith('string') ]
cols = rain_df.columns

In [27]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
label_stringIdx = StringIndexer(inputCol = 'RainTomorrow', outputCol = 'label')
stages += [label_stringIdx]
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericColumns
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [28]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(rain_df)
rain_df = pipelineModel.transform(rain_df)
selectedCols = ['label', 'features'] + cols
rain_df = rain_df.select(selectedCols)
rain_df.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- Location: string (nullable = true)
 |-- Rainfall: integer (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: integer (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: integer (nullable = true)
 |-- WindSpeed3pm: integer (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |-- MinTemp_imputed: integer (nullable = true)
 |-- MaxTemp_imputed: integer (nullable = true)
 |-- Temp9am_imputed: integer (nullable = true)
 |-- Pressure9am_imputed: integer (nullable = true)
 |-- Temp3pm_imputed: integer (nullable = true)
 |-- Humidity3pm_imputed: integer (nullable = true)
 |-- Humidity9am_imputed: integer (nullable = true)
 |-- Pressure3pm_imputed: integer (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)



In [48]:
rain_df.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|96318|
|  1.0|27392|
+-----+-----+



In [29]:
Majorityclass = rain_df.filter(col("label") == 0)
Minorityclass = rain_df.filter(col("label") == 1)
imbalance= int(Majorityclass.count()/Minorityclass.count())

Majority_sam = Majorityclass.sample(False, 1/imbalance)
Undersam = Majority_sam.unionAll(Minorityclass)
Undersam.show()
print(Undersam.count())
Undersam.groupBy('label').count().show()


+-----+--------------------+-------------+--------+-----------+-------------+----------+----------+------------+------------+------------+---------------+---------------+---------------+-------------------+---------------+-------------------+-------------------+-------------------+----+-----+---+
|label|            features|     Location|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|RainTomorrow|MinTemp_imputed|MaxTemp_imputed|Temp9am_imputed|Pressure9am_imputed|Temp3pm_imputed|Humidity3pm_imputed|Humidity9am_imputed|Pressure3pm_imputed|Year|Month|Day|
+-----+--------------------+-------------+--------+-----------+-------------+----------+----------+------------+------------+------------+---------------+---------------+---------------+-------------------+---------------+-------------------+-------------------+-------------------+----+-----+---+
|  0.0|(155,[36,64,79,91...|       Albury|       0|        NNE|           31|        SE|       SSE|       

In [30]:
Undersam.groupBy('Location').count().show(100)

+----------------+-----+
|        Location|count|
+----------------+-----+
|          Cairns| 1574|
|   NorfolkIsland| 1519|
|         Bendigo| 1234|
|         Walpole| 1400|
|        Canberra| 1277|
|         Woomera| 1095|
|        Adelaide| 1303|
|           Cobar| 1234|
|   SydneyAirport| 1460|
|    PerthAirport| 1385|
|      Wollongong| 1378|
|     Williamtown| 1085|
|           Moree| 1123|
|         Mildura| 1217|
|        Portland| 1674|
|      SalmonGums| 1295|
|        Brisbane| 1453|
|          Sydney| 1164|
|           Perth| 1475|
|            Sale| 1276|
|   BadgerysCreek| 1118|
|          Hobart| 1555|
|     Tuggeranong| 1073|
|        Ballarat| 1417|
|       GoldCoast| 1440|
|MelbourneAirport| 1412|
|        Dartmoor| 1355|
|            Nhil|  644|
|      PearceRAAF| 1129|
|          Albury| 1168|
|     Witchcliffe| 1426|
|      WaggaWagga| 1307|
|          Darwin| 1587|
|           Uluru|  580|
|       Nuriootpa| 1317|
|    CoffsHarbour| 1362|
|       Melbourne| 1067|


In [31]:
train_df, test_df = Undersam.randomSplit([0.8, 0.2],)
print("Training Dataset Count: " + str(train_df.count()))
print("Test Dataset Count: " + str(test_df.count()))

Training Dataset Count: 47548
Test Dataset Count: 11986


In [49]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
lr = (LogisticRegression(featuresCol = 'features', labelCol = 'label',maxIter=100))
lrModel = lr.fit(train_df)
lrpredicted=lrModel.transform(test_df)
evaluator= BinaryClassificationEvaluator()
print('Test area under AOC',evaluator.evaluate(lrpredicted))
accuracy=lrpredicted.filter(lrpredicted.label==lrpredicted.prediction).count()/float(lrpredicted.count())
print('Accuracy:',accuracy)
y_true=lrpredicted.select(['label']).collect()
y_pred=lrpredicted.select(['prediction']).collect()
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true,y_pred))

Test area under AOC 0.9999997478113174
Accuracy: 1.0
[[6471    0]
 [   0 5515]]
